In [1]:
import os
#import jsondim
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import *
from sklearn import metrics
import numpy as np
from collections import defaultdict
import sys

In [3]:
# class LSTMMax(nn.Module):
#     def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
#         super(LSTMMax, self).__init__()
#         self.embed_size=embed_size
#         self.rnn_size=rnn_size
#         self.cond_vocab_size=cond_vocab_size
#         self.cond_seq_len=cond_seq_len
#         self.proc_vocab_size=proc_vocab_size
#         self.proc_seq_len=proc_seq_len
#         self.med_vocab_size=med_vocab_size
#         self.med_seq_len=med_seq_len
#         self.lab_vocab_size=lab_vocab_size
#         self.lab_seq_len=lab_seq_len
#         self.time_vocab_size=time_vocab_size
#         self.census_vocab_size=census_vocab_size
#         self.batch_size=batch_size
#         self.padding_idx = 0
#         self.modalities=5
#         self.device=device
#         self.build()
        
#     def build(self):
#         self.condEmbed=nn.Embedding(self.cond_vocab_size,self.embed_size,self.padding_idx)
#         self.procEmbed=nn.Embedding(self.proc_vocab_size,self.embed_size,self.padding_idx)
#         self.medEmbed=nn.Embedding(self.med_vocab_size,self.embed_size,self.padding_idx)
#         self.labEmbed=nn.Embedding(self.lab_vocab_size,self.embed_size,self.padding_idx)
#         self.censusEmbed=nn.Embedding(self.census_vocab_size,self.embed_size,self.padding_idx)
        
#         self.condRnn = nn.LSTM(input_size=self.embed_size*3,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
#         self.procRnn = nn.LSTM(input_size=self.embed_size*3,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
#         self.medRnn = nn.LSTM(input_size=self.embed_size*3,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
#         self.labRnn = nn.LSTM(input_size=self.embed_size*3,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
        
#         self.cond_max = nn.AdaptiveMaxPool1d(1, True)
#         self.proc_max = nn.AdaptiveMaxPool1d(1, True)
#         self.med_max = nn.AdaptiveMaxPool1d(1, True)
#         self.lab_max = nn.AdaptiveMaxPool1d(1, True)
#         self.census_max = nn.AdaptiveMaxPool1d(1, True)
        
#         self.cond_fc=nn.Linear(self.rnn_size, 1, False)
#         self.proc_fc=nn.Linear(self.rnn_size, 1, False)
#         self.med_fc=nn.Linear(self.rnn_size, 1, False)
#         self.lab_fc=nn.Linear(self.rnn_size, 1, False)
#         self.census_fc=nn.Linear(self.embed_size, 1, False)
        
#         #self.fc=nn.Linear(self.modalities, 1, False)
        
#         self.sig = nn.Sigmoid()
#         self.timeEmbed=nn.Embedding(self.time_vocab_size,self.embed_size,self.padding_idx)
#         self.cond_drop = nn.Dropout(0.2)
#         self.proc_drop = nn.Dropout(0.4)
#         self.med_drop = nn.Dropout(0.4)
#         self.lab_drop = nn.Dropout(0.4)
#         #self.census_drop = nn.Dropout()
        
#     def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, lab_lengths,census,contrib):
        
#         #print(conds.shape)
#         cond_time=conds[1]
#         conds=conds[0]
#         proc_time=procs[1]
#         procs=procs[0]
#         med_time=meds[1]
#         meds=meds[0]
#         lab_time=labs[1]
#         labs=labs[0]
        
        
#         #print(cond_time[0])
#         #cond_time=np.diff(cond_time)
#         #print(cond_time[0])
#         #print(torch.diff(cond_time,dim=1)[0])
#         #print(time.shape)
#         #initialize hidden and cell state
#         h_0, c_0 = self.init_hidden()
#         #print("h",h_0.shape)
        
#         h_0, c_0, cond_time, conds = h_0.to(self.device), c_0.to(self.device),cond_time.to(self.device),conds.to(self.device)
#         proc_time, procs =proc_time.to(self.device),procs.to(self.device)
#         med_time, meds =med_time.to(self.device),meds.to(self.device)
#         lab_time, labs =lab_time.to(self.device),labs.to(self.device)
#         census=census.to(self.device)
        
#         #Embedd all sequences
#         #print("embedded")
#         conds=self.condEmbed(conds)
#         cond_time=self.timeEmbed(cond_time)
#         cond_time_sin=torch.sin(cond_time)
#         cond_time_cos=torch.cos(cond_time)
#         #print(cond_time.shape)
#         #print(conds.shape)
#         procs=self.procEmbed(procs)
#         #print("time")
#         proc_time=self.timeEmbed(proc_time)
#         proc_time_sin=torch.sin(proc_time)
#         proc_time_cos=torch.cos(proc_time)
#         #print("embedded")
#         meds=self.medEmbed(meds)
#         med_time=self.timeEmbed(med_time)
#         med_time_sin=torch.sin(med_time)
#         med_time_cos=torch.cos(med_time)
#         #print("embedded")
#         labs=self.labEmbed(labs)
#         lab_time=self.timeEmbed(lab_time)
#         lab_time_sin=torch.sin(lab_time)
#         lab_time_cos=torch.cos(lab_time)
#         #print("embedded",conds.shape)
#         #print(conds[0])
#         census=self.censusEmbed(census)
#         #print("embedded",census.shape)
        
        
#         conds=torch.cat((conds,cond_time_sin,cond_time_cos),dim=2)
#         #print(conds.shape)
#         procs=torch.cat((procs,proc_time_sin,proc_time_cos),dim=2)
#         #print(procs.shape)
#         meds=torch.cat((meds,med_time_sin,med_time_cos),dim=2)
#         #print(meds.shape)
#         labs=torch.cat((labs,lab_time_sin,lab_time_cos),dim=2)
#         #print(labs.shape)
#         #print("padding")
#         #Pack all sequences
#         conds_pack = torch.nn.utils.rnn.pack_padded_sequence(conds, cond_lengths, batch_first=True,enforce_sorted=False)
#         #print("cond")
#         procs_pack = torch.nn.utils.rnn.pack_padded_sequence(procs, proc_lengths, batch_first=True,enforce_sorted=False)
#         #print("proc")
#         meds_pack = torch.nn.utils.rnn.pack_padded_sequence(meds, med_lengths, batch_first=True,enforce_sorted=False)
#         #print("meds")
#         labs_pack = torch.nn.utils.rnn.pack_padded_sequence(labs, lab_lengths, batch_first=True,enforce_sorted=False)
#         #print("labs")
#         #Run through LSTM
#         cond_output, (cond_h_n, cond_c_n)=self.condRnn( conds_pack, (h_0, c_0))
#         #print("cond")
#         proc_output, (proc_h_n, proc_c_n)=self.procRnn( procs_pack, (h_0, c_0))
#         med_output, (med_h_n, med_c_n)=self.medRnn( meds_pack, (h_0, c_0))
#         lab_output, (lab_h_n, lab_c_n)=self.labRnn( labs_pack, (h_0, c_0))
#         #print("h_n",h_n.shape)
       
#         #unpack sequence
#         cond_output, _ = torch.nn.utils.rnn.pad_packed_sequence(cond_output, batch_first=True)
#         proc_output, _ = torch.nn.utils.rnn.pad_packed_sequence(proc_output, batch_first=True)
#         med_output, _ = torch.nn.utils.rnn.pad_packed_sequence(med_output, batch_first=True)
#         lab_output, _ = torch.nn.utils.rnn.pad_packed_sequence(lab_output, batch_first=True)
#         #print("upack")
#         cond_output=self.cond_drop(cond_output)
#         proc_output=self.proc_drop(proc_output)
#         med_output=self.med_drop(med_output)
#         lab_output=self.lab_drop(lab_output)
        
#         cond_output=F.relu(cond_output)
#         proc_output=F.relu(proc_output)
#         med_output=F.relu(med_output)
#         lab_output=F.relu(lab_output)
        
        
#         #print("output",cond_output.shape)
#         #print("hi")
#         cond_output=cond_output.view(cond_output.shape[0],cond_output.shape[2],cond_output.shape[1])
#         proc_output=proc_output.view(proc_output.shape[0],proc_output.shape[2],proc_output.shape[1])
#         med_output=med_output.view(med_output.shape[0],med_output.shape[2],med_output.shape[1])
#         lab_output=lab_output.view(lab_output.shape[0],lab_output.shape[2],lab_output.shape[1])
#         census_output=census.view(census.shape[0],census.shape[2],census.shape[1])
#         #print("before maxpool",census_output.shape)
#         census_output=F.relu(census_output)
        
#         cond_pool, cond_indices = self.cond_max(cond_output)
#         proc_pool, proc_indices = self.proc_max(proc_output)
#         med_pool, med_indices = self.med_max(med_output)
#         lab_pool, lab_indices = self.lab_max(lab_output)
#         census_pool, census_indices = self.census_max(census_output)
#         #print("indices",cond_indices.shape)
#         #print(cond_indices[0])
        
        
        
#         cond_pool = torch.squeeze(cond_pool)
#         cond_indices = torch.squeeze(cond_indices)
#         proc_pool = torch.squeeze(proc_pool)
#         proc_indices = torch.squeeze(proc_indices)
#         med_pool = torch.squeeze(med_pool)
#         med_indices = torch.squeeze(med_indices)
#         lab_pool = torch.squeeze(lab_pool)
#         lab_indices = torch.squeeze(lab_indices)
#         census_pool = torch.squeeze(census_pool)
#         census_indices = torch.squeeze(census_indices)
#         #print("max output",cond_pool.shape)
#         #print(cond_pool[0])
#         #cond_output = torch.cat((cond_output,torch.zeros(cond_output.shape[0],int(self.cond_seq_len-cond_output.shape[1]))),1)
#         #print("max output",cond_output.shape)
#         #print(cond_output[0])
        
#         cond_out1 = self.cond_fc(cond_pool)
#         proc_out1 = self.proc_fc(proc_pool)
#         med_out1 = self.med_fc(med_pool)
#         lab_out1 = self.lab_fc(lab_pool)
#         census_out1 = self.census_fc(census_pool)
        
#         #print(cond_out1.shape)
#         #print(proc_out1.shape)
#         #print(med_out1.shape)
#         #print(lab_out1.shape)
#         #print(census_out1.shape)
#         #out1=torch.cat((cond_out1,proc_out1,med_out1,lab_out1,census_out1),1)
#         out1=torch.add(cond_out1, proc_out1)
#         out1=torch.add(out1, med_out1)
#         out1=torch.add(out1, lab_out1)
#         out1=torch.add(out1, census_out1)
#         #print(out1.shape)
#         #out1=self.fc(out1)
#         sigout1 = self.sig(out1)
#         #print("sig out",out1.shape)
#         #print(out1[0])
        
#         if contrib:
#             cond_pool = cond_pool.data.cpu().numpy()
#             proc_pool = proc_pool.data.cpu().numpy()
#             med_pool = med_pool.data.cpu().numpy()
#             lab_pool = lab_pool.data.cpu().numpy()
#             census_pool = census_pool.data.cpu().numpy()
#             #print(cond_pool.shape)
#             assert cond_pool.shape == cond_indices.shape
#             cond_fc_weight = self.cond_fc.weight.data.cpu().numpy().reshape(-1)
#             proc_fc_weight = self.proc_fc.weight.data.cpu().numpy().reshape(-1)
#             med_fc_weight = self.med_fc.weight.data.cpu().numpy().reshape(-1)
#             lab_fc_weight = self.lab_fc.weight.data.cpu().numpy().reshape(-1)
#             census_fc_weight = self.census_fc.weight.data.cpu().numpy().reshape(-1)
#             #print("logit_weight",fc_weight.shape)
#             cond_contri = np.zeros((self.batch_size, self.cond_seq_len))
#             proc_contri = np.zeros((self.batch_size, self.proc_seq_len))
#             med_contri = np.zeros((self.batch_size, self.med_seq_len))
#             lab_contri = np.zeros((self.batch_size, self.lab_seq_len))
#             census_contri = np.zeros((self.batch_size, self.census_vocab_size))
#             #print("contrib",cond_contri.shape)
#             for i in range(cond_pool.shape[0]):
#                 for j in range(cond_pool.shape[1]):
#                     con = cond_pool[i,j] * cond_fc_weight[j]
#                     idx = cond_indices[i, j]
#                     cond_contri[i,idx] += con
#             #print("contrib",cond_contri.shape)        
#             for i in range(proc_pool.shape[0]):
#                 for j in range(proc_pool.shape[1]):
#                     con = proc_pool[i,j] * proc_fc_weight[j]
#                     idx = proc_indices[i, j]
#                     proc_contri[i,idx] += con
#             #print("contrib",cond_contri.shape)        
#             for i in range(med_pool.shape[0]):
#                 for j in range(med_pool.shape[1]):
#                     con = med_pool[i,j] * med_fc_weight[j]
#                     idx = med_indices[i, j]
#                     med_contri[i,idx] += con
#             #print("contrib",cond_contri.shape)        
#             for i in range(lab_pool.shape[0]):
#                 for j in range(lab_pool.shape[1]):
#                     con = lab_pool[i,j] * lab_fc_weight[j]
#                     idx = lab_indices[i, j]
#                     lab_contri[i,idx] += con
#             #print("contrib",cond_contri.shape)
#             for i in range(census_pool.shape[0]):
#                 for j in range(census_pool.shape[1]):
#                     con = census_pool[i,j] * census_fc_weight[j]
#                     idx = census_indices[i, j]
#                     census_contri[i,idx] += con
#             #print("contrib",cond_contri[0])
#             cond_output = cond_output.data.cpu().numpy()
#             return sigout1,out1, cond_contri,proc_contri,med_contri,lab_contri,census_contri,cond_pool,cond_fc_weight,cond_indices,conds
#         else:
#             return sigout1
            
        
#     def init_hidden(self):
#         # initialize the hidden state and the cell state to zeros
#         h=torch.zeros(1,self.batch_size, self.rnn_size)
#         c=torch.zeros(1,self.batch_size, self.rnn_size)

# #         if self.hparams.on_gpu:
# #             hidden_a = hidden_a.cuda()
# #             hidden_b = hidden_b.cuda()

#         h = Variable(h)
#         c = Variable(c)

#         return (h, c)    

In [ ]:
class CNNStatic(nn.Module):
    def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,bmi_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(CNNStatic, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.build()
        
    def build(self):
        self.cond=CodeCNN(self.device,self.embed_size,self.rnn_size,self.cond_vocab_size,self.cond_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.proc=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.proc_vocab_size,self.proc_seq_len,self.time_vocab_size,self.batch_size)
        self.med=CodeCNN(self.device,self.embed_size,self.rnn_size,self.med_vocab_size,self.med_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.lab=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.lab_vocab_size,self.lab_seq_len,self.time_vocab_size,self.batch_size)
        self.bmi=CodeCNN(self.device,self.embed_size,self.rnn_size,None,self.bmi_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=True)
        
        self.cond_fc=nn.Linear(self.rnn_size, 1, False)
        #self.proc_fc=nn.Linear(self.rnn_size, 1, False)
        self.med_fc=nn.Linear(self.rnn_size, 1, False)
        #self.lab_fc=nn.Linear(self.rnn_size, 1, False)
        self.bmi_fc=nn.Linear(self.rnn_size, 1, False)
        
        
        self.sig = nn.Sigmoid()
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, lab_lengths,bmi, bmi_lengths,census,contrib):        
        
        cond_pool,cond_indices =self.cond(conds, cond_lengths)
        #print("cond")
        #proc_pool,proc_indices = self.proc(procs, proc_lengths)      
        med_pool,med_indices = self.med(meds, med_lengths)    
        #print("med")
        #lab_pool,lab_indices = self.lab(labs, lab_lengths)  
        bmi_pool,bmi_indices =self.bmi(bmi, bmi_lengths)
        #print("bmi")
        

        #print("hi")
        cond_out1 = self.cond_fc(cond_pool)
        #proc_out1 = self.proc_fc(proc_pool)
        med_out1 = self.med_fc(med_pool)
        #lab_out1 = self.lab_fc(lab_pool)
        bmi_out1 = self.bmi_fc(bmi_pool)
        
        
        #print(cond_out1.shape)
        #print(proc_out1.shape)
        #print(med_out1.shape)
        #print(lab_out1.shape)
        #print(census_out1.shape)
        #out1=torch.cat((cond_out1,proc_out1,med_out1,lab_out1,census_out1),1)
        #out1=torch.add(cond_out1, med_out1)
        
        #out1=torch.add(out1, lab_out1)
        out1=torch.add(cond_out1, bmi_out1)
        #print("hi")
        out1=torch.add(out1, med_out1)
        #print("hi")
        #print(out1.shape)
        #print(census_out1.shape)
        
        #print("hi")
        #print(out1)
        #out1=self.fc(out1)
        sigout1 = self.sig(out1)
        #print("sig out",out1.shape)
        #print(out1[0])
        #print("hi")
        
        return sigout1
        
            

In [ ]:
class CNNMax(nn.Module):
    def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,bmi_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(CNNMax, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.build()
        
    def build(self):
        self.cond=CodeCNN(self.device,self.embed_size,self.rnn_size,self.cond_vocab_size,self.cond_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.proc=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.proc_vocab_size,self.proc_seq_len,self.time_vocab_size,self.batch_size)
        self.med=CodeCNN(self.device,self.embed_size,self.rnn_size,self.med_vocab_size,self.med_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.lab=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.lab_vocab_size,self.lab_seq_len,self.time_vocab_size,self.batch_size)
        self.bmi=CodeCNN(self.device,self.embed_size,self.rnn_size,None,self.bmi_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=True)
        
        self.censusEmbed=nn.Embedding(self.census_vocab_size,self.embed_size,self.padding_idx)             
        self.census_max = nn.AdaptiveMaxPool1d(1, True)        
        
        
        self.cond_fc=nn.Linear(self.rnn_size, 1, False)
        #self.proc_fc=nn.Linear(self.rnn_size, 1, False)
        self.med_fc=nn.Linear(self.rnn_size, 1, False)
        #self.lab_fc=nn.Linear(self.rnn_size, 1, False)
        self.bmi_fc=nn.Linear(self.rnn_size, 1, False)
        self.census_fc=nn.Linear(self.embed_size, 1, False)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, lab_lengths,bmi, bmi_lengths,census,contrib):        
        
        cond_pool,cond_indices =self.cond(conds, cond_lengths)
        #print("cond")
        #proc_pool,proc_indices = self.proc(procs, proc_lengths)      
        med_pool,med_indices = self.med(meds, med_lengths)    
        #print("med")
        #lab_pool,lab_indices = self.lab(labs, lab_lengths)  
        bmi_pool,bmi_indices =self.bmi(bmi, bmi_lengths)
        #print("bmi")
        census=census.to(self.device)              
        census=self.censusEmbed(census)     
        census_output=census.view(census.shape[0],census.shape[2],census.shape[1])
        census_output=F.relu(census_output)
        census_pool, census_indices = self.census_max(census_output)               
        census_pool,census_indices  = torch.squeeze(census_pool), torch.squeeze(census_indices)

        #print("hi")
        cond_out1 = self.cond_fc(cond_pool)
        #proc_out1 = self.proc_fc(proc_pool)
        med_out1 = self.med_fc(med_pool)
        #lab_out1 = self.lab_fc(lab_pool)
        bmi_out1 = self.bmi_fc(bmi_pool)
        census_out1 = self.census_fc(census_pool)
        
        #print(cond_out1.shape)
        #print(proc_out1.shape)
        #print(med_out1.shape)
        #print(lab_out1.shape)
        #print(census_out1.shape)
        #out1=torch.cat((cond_out1,proc_out1,med_out1,lab_out1,census_out1),1)
        #out1=torch.add(cond_out1, med_out1)
        
        #out1=torch.add(out1, lab_out1)
        out1=torch.add(cond_out1, bmi_out1)
        #print("hi")
        out1=torch.add(out1, med_out1)
        #print("hi")
        #print(out1.shape)
        #print(census_out1.shape)
        out1=torch.add(out1, census_out1)
        #print("hi")
        #print(out1)
        #out1=self.fc(out1)
        sigout1 = self.sig(out1)
        #print("sig out",out1.shape)
        #print(out1[0])
        #print("hi")
        if contrib:
            cond_pool = cond_pool.data.cpu().numpy()
            #proc_pool = proc_pool.data.cpu().numpy()
            #med_pool = med_pool.data.cpu().numpy()
            #lab_pool = lab_pool.data.cpu().numpy()
            bmi_pool = bmi_pool.data.cpu().numpy()
            census_pool = census_pool.data.cpu().numpy()
            #print(cond_pool.shape)
            assert cond_pool.shape == cond_indices.shape
            cond_fc_weight = self.cond_fc.weight.data.cpu().numpy().reshape(-1)
            #proc_fc_weight = self.proc_fc.weight.data.cpu().numpy().reshape(-1)
            #med_fc_weight = self.med_fc.weight.data.cpu().numpy().reshape(-1)
            #lab_fc_weight = self.lab_fc.weight.data.cpu().numpy().reshape(-1)
            bmi_fc_weight = self.bmi_fc.weight.data.cpu().numpy().reshape(-1)
            census_fc_weight = self.census_fc.weight.data.cpu().numpy().reshape(-1)
            #print("logit_weight",fc_weight.shape)
            cond_contri = np.zeros((self.batch_size, self.cond_seq_len))
            #proc_contri = np.zeros((self.batch_size, self.proc_seq_len))
            #med_contri = np.zeros((self.batch_size, self.med_seq_len))
            #lab_contri = np.zeros((self.batch_size, self.lab_seq_len))
            bmi_contri = np.zeros((self.batch_size, self.bmi_seq_len))
            census_contri = np.zeros((self.batch_size, self.census_vocab_size))
            #print("contrib",cond_contri.shape)
            for i in range(cond_pool.shape[0]):
                for j in range(cond_pool.shape[1]):
                    con = cond_pool[i,j] * cond_fc_weight[j]
                    idx = cond_indices[i, j]
                    cond_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(proc_pool.shape[0]):
#                for j in range(proc_pool.shape[1]):
#                    con = proc_pool[i,j] * proc_fc_weight[j]
#                    idx = proc_indices[i, j]
#                    proc_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#             for i in range(med_pool.shape[0]):
#                 for j in range(med_pool.shape[1]):
#                     con = med_pool[i,j] * med_fc_weight[j]
#                     idx = med_indices[i, j]
#                     med_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(lab_pool.shape[0]):
#                for j in range(lab_pool.shape[1]):
#                    con = lab_pool[i,j] * lab_fc_weight[j]
#                    idx = lab_indices[i, j]
#                    lab_contri[i,idx] += con
            for i in range(bmi_pool.shape[0]):
                for j in range(bmi_pool.shape[1]):
                    con = bmi_pool[i,j] * bmi_fc_weight[j]
                    idx = bmi_indices[i, j]
                    bmi_contri[i,idx] += con
            #print("contrib",cond_contri.shape)
            for i in range(census_pool.shape[0]):
                for j in range(census_pool.shape[1]):
                    con = census_pool[i,j] * census_fc_weight[j]
                    idx = census_indices[i, j]
                    census_contri[i,idx] += con
            #print("contrib",cond_contri[0])
            #cond_output = cond_output.data.cpu().numpy()
            #return sigout1,out1, cond_contri,proc_contri,med_contri,lab_contri,census_contri,cond_pool,cond_fc_weight,cond_indices
            return sigout1,out1, cond_contri,bmi_contri,census_contri,cond_pool,cond_fc_weight,cond_indices
        else:
            return sigout1
        
            

In [ ]:
class CodeCNN(nn.Module):
    def __init__(self,device,embed_size,rnn_size,code_vocab_size,code_seq_len,time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeCNN, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
        if self.bmi_flag:
            self.conv1 = nn.Conv1d(self.embed_size+1,self.rnn_size, kernel_size = 3, stride = 1, padding =0)
        else:
            self.codeEmbed=nn.Embedding(self.code_vocab_size,self.embed_size,self.padding_idx)
            self.conv1 = nn.Conv1d(self.embed_size * 2,self.rnn_size, kernel_size = 3, stride = 1, padding =0)
        self.bn1 = nn.BatchNorm1d(self.rnn_size)
        self.maxpool1 = nn.AdaptiveMaxPool1d(1, True)
        self.dropout1 = nn.Dropout(0.2)
        
        self.timeEmbed = nn.Embedding(self.time_vocab_size, self.embed_size). \
            from_pretrained(embeddings=self._init_posi_embedding(self.time_vocab_size, self.embed_size))

    def forward(self, code,code_lengths):
        #print(conds.shape)
        code_time=code[1]
        code=code[0]
        
        #initialize hidden and cell state
        h_0, c_0 = self.init_hidden()
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        #Embedd all sequences
        #print("embedded")
        if self.bmi_flag:
            code=code.unsqueeze(2)
            #print("code",code.shape)
            #print(self.code_seq_len)
            #code=code.type(torch.LongTensor)
        else:
            code=self.codeEmbed(code)
        
        #code_time=code_time.type(torch.LongTensor)
        #h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)
        code_time=self.timeEmbed(code_time)
        #print("hi")
        
        code=code.type(torch.FloatTensor)
        code_time=code_time.type(torch.FloatTensor)
        code_time, code = code_time.to(self.device),code.to(self.device)

        code=torch.cat((code,code_time),dim=2)
        #code = self.dropout1(code)
        code=code.permute(0,2,1)
        #print("input",code.shape)
        #code=self.code_drop(code) 
        
        #Run through LSTM
        cond_output = self.conv1(code)
        #print("output",cond_output.shape)
        cond_output = self.bn1(cond_output)
        #print("output",cond_output.shape)
        cond_output = F.relu(cond_output)
        #print("output",cond_output.shape)

        code_pool, code_indices = self.maxpool1(cond_output)
        #print("output",code_pool.shape)
        
        
        code_pool = torch.squeeze(code_pool)
        code_indices = torch.squeeze(code_indices)
        
        return code_pool,code_indices
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


In [ ]:
class LSTMStatic(nn.Module):
    def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,bmi_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(LSTMStatic, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.build()
        
    def build(self):
        self.cond=CodeBase(self.device,self.embed_size,self.rnn_size,self.cond_vocab_size,self.cond_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.proc=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.proc_vocab_size,self.proc_seq_len,self.time_vocab_size,self.batch_size)
        self.med=CodeBase(self.device,self.embed_size,self.rnn_size,self.med_vocab_size,self.med_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.lab=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.lab_vocab_size,self.lab_seq_len,self.time_vocab_size,self.batch_size)
        self.bmi=CodeBase(self.device,self.embed_size,self.rnn_size,None,self.bmi_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=True)
        
        #self.censusEmbed=nn.Embedding(self.census_vocab_size,self.embed_size,self.padding_idx)                           
        
        self.cond_fc=nn.Linear(self.rnn_size, 1, False)
        #self.proc_fc=nn.Linear(self.rnn_size, 1, False)
        self.med_fc=nn.Linear(self.rnn_size, 1, False)
        #self.lab_fc=nn.Linear(self.rnn_size, 1, False)
        self.bmi_fc=nn.Linear(self.rnn_size, 1, False)
        #self.census_fc=nn.Linear(self.census_vocab_size*self.embed_size, 1, False)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, lab_lengths,bmi, bmi_lengths,census,contrib):        
        
        cond_h_n =self.cond(conds, cond_lengths)
        #print("cond")
        #proc_pool,proc_indices = self.proc(procs, proc_lengths)      
        med_h_n = self.med(meds, med_lengths)    
        #print("med")
        #lab_pool,lab_indices = self.lab(labs, lab_lengths)  
        bmi_h_n =self.bmi(bmi, bmi_lengths)
        #print("bmi")
        

        #print("hi")
        cond_out1 = self.cond_fc(cond_h_n)
        #proc_out1 = self.proc_fc(proc_pool)
        med_out1 = self.med_fc(med_h_n)
        #lab_out1 = self.lab_fc(lab_pool)
        bmi_out1 = self.bmi_fc(bmi_h_n)
        #print("hi")
        
        #print("hi")
        #out1=torch.add(out1, lab_out1)
        out1=torch.add(cond_out1, bmi_out1)
        #print("hi")
        out1=torch.add(out1, med_out1)
        #print("hi")
        #print(out1.shape)
        #print(census_out1.shape)
        #print("hi")
        #print(out1)
        #out1=self.fc(out1)
        sigout1 = self.sig(out1)
        #print("sig out",out1.shape)
        #print(out1[0])
        #print("hi")
        
        return sigout1
        
            

In [ ]:
class LSTMBase(nn.Module):
    def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,bmi_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(LSTMBase, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.build()
        
    def build(self):
        self.cond=CodeBase(self.device,self.embed_size,self.rnn_size,self.cond_vocab_size,self.cond_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.proc=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.proc_vocab_size,self.proc_seq_len,self.time_vocab_size,self.batch_size)
        self.med=CodeBase(self.device,self.embed_size,self.rnn_size,self.med_vocab_size,self.med_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.lab=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.lab_vocab_size,self.lab_seq_len,self.time_vocab_size,self.batch_size)
        self.bmi=CodeBase(self.device,self.embed_size,self.rnn_size,None,self.bmi_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=True)
        
        self.censusEmbed=nn.Embedding(self.census_vocab_size,self.embed_size,self.padding_idx)                           
        
        self.cond_fc=nn.Linear(self.rnn_size, 1, False)
        #self.proc_fc=nn.Linear(self.rnn_size, 1, False)
        self.med_fc=nn.Linear(self.rnn_size, 1, False)
        #self.lab_fc=nn.Linear(self.rnn_size, 1, False)
        self.bmi_fc=nn.Linear(self.rnn_size, 1, False)
        self.census_fc=nn.Linear(self.census_vocab_size*self.embed_size, 1, False)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, lab_lengths,bmi, bmi_lengths,census,contrib):        
        
        cond_h_n =self.cond(conds, cond_lengths)
        #print("cond")
        #proc_pool,proc_indices = self.proc(procs, proc_lengths)      
        med_h_n = self.med(meds, med_lengths)    
        #print("med")
        #lab_pool,lab_indices = self.lab(labs, lab_lengths)  
        bmi_h_n =self.bmi(bmi, bmi_lengths)
        #print("bmi")
        census=census.to(self.device)              
        census=self.censusEmbed(census)     
        census_output=torch.reshape(census,(census.shape[0],-1))
        #print("census",census_output.shape)

        #print("hi")
        cond_out1 = self.cond_fc(cond_h_n)
        #proc_out1 = self.proc_fc(proc_pool)
        med_out1 = self.med_fc(med_h_n)
        #lab_out1 = self.lab_fc(lab_pool)
        bmi_out1 = self.bmi_fc(bmi_h_n)
        #print("hi")
        census_out1 = self.census_fc(census_output)
        #print("hi")
        #out1=torch.add(out1, lab_out1)
        out1=torch.add(cond_out1, bmi_out1)
        #print("hi")
        out1=torch.add(out1, med_out1)
        #print("hi")
        #print(out1.shape)
        #print(census_out1.shape)
        out1=torch.add(out1, census_out1)
        #print("hi")
        #print(out1)
        #out1=self.fc(out1)
        sigout1 = self.sig(out1)
        #print("sig out",out1.shape)
        #print(out1[0])
        #print("hi")
        
        return sigout1
        
            

In [ ]:
class CodeBase(nn.Module):
    def __init__(self,device,embed_size,rnn_size,code_vocab_size,code_seq_len,time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeBase, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
        if self.bmi_flag:
            self.codeRnn = nn.LSTM(input_size=self.embed_size + 1,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
        else:
            self.codeEmbed=nn.Embedding(self.code_vocab_size,self.embed_size,self.padding_idx)
            self.codeRnn = nn.LSTM(input_size=self.embed_size*2,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
            
        self.code_drop = nn.Dropout(0.2)
        
        self.timeEmbed = nn.Embedding(self.time_vocab_size, self.embed_size). \
            from_pretrained(embeddings=self._init_posi_embedding(self.time_vocab_size, self.embed_size))

    def forward(self, code,code_lengths):
        #print(conds.shape)
        code_time=code[1]
        code=code[0]
        
        #initialize hidden and cell state
        h_0, c_0 = self.init_hidden()
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        #Embedd all sequences
        #print("embedded")
        if self.bmi_flag:
            code=code.unsqueeze(2)
            #print("code",code.shape)
            #print(self.code_seq_len)
            #code=code.type(torch.LongTensor)
        else:
            code=self.codeEmbed(code)
        
        #code_time=code_time.type(torch.LongTensor)
        #h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)
        code_time=self.timeEmbed(code_time)
        #print("hi")
        #print(code_time.shape)
        code=code.type(torch.FloatTensor)
        code_time=code_time.type(torch.FloatTensor)
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        code=torch.cat((code,code_time),dim=2)
        #print("hi")
        #code=self.code_drop(code) 
        #Pack all sequences
        code_pack = torch.nn.utils.rnn.pack_padded_sequence(code, code_lengths, batch_first=True,enforce_sorted=False)
        #print("hi")
        #Run through LSTM
        code_output, (code_h_n, code_c_n)=self.codeRnn(code_pack, (h_0, c_0))
        #print("hi")
        #print("output",code_h_n.shape)
        code_h_n=code_h_n.squeeze()
                      
        #print("output",code_h_n.shape)
        #print("hi")
        
        
        
        return code_h_n
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


In [ ]:
class LSTMAttn(nn.Module):
    def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,bmi_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(LSTMAttn, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.build()
        
    def build(self):
        self.cond=CodeAttn(self.device,self.embed_size,self.rnn_size,self.cond_vocab_size,self.cond_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.proc=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.proc_vocab_size,self.proc_seq_len,self.time_vocab_size,self.batch_size)
        self.med=CodeAttn(self.device,self.embed_size,self.rnn_size,self.med_vocab_size,self.med_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.lab=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.lab_vocab_size,self.lab_seq_len,self.time_vocab_size,self.batch_size)
        self.bmi=CodeAttn(self.device,self.embed_size,self.rnn_size,None,self.bmi_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=True)
        
        self.censusEmbed=nn.Embedding(self.census_vocab_size,self.embed_size,self.padding_idx)             
        self.census_max = nn.AdaptiveMaxPool1d(1, True)        
        
        
        self.cond_fc=nn.Linear(self.rnn_size, 1, False)
        #self.proc_fc=nn.Linear(self.rnn_size, 1, False)
        self.med_fc=nn.Linear(self.rnn_size, 1, False)
        #self.lab_fc=nn.Linear(self.rnn_size, 1, False)
        self.bmi_fc=nn.Linear(self.rnn_size, 1, False)
        self.census_fc=nn.Linear(self.census_vocab_size*self.embed_size, 1, False)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, lab_lengths,bmi, bmi_lengths,census,contrib):        
        
        cond_context =self.cond(conds, cond_lengths)
        #print("cond")
        #proc_pool,proc_indices = self.proc(procs, proc_lengths)      
        med_context = self.med(meds, med_lengths)    
        #print("med")
        #lab_pool,lab_indices = self.lab(labs, lab_lengths)  
        bmi_context =self.bmi(bmi, bmi_lengths)
        #print("bmi")
        census=census.to(self.device)              
        census=self.censusEmbed(census)     
        census_output=torch.reshape(census,(census.shape[0],-1))

        #print("hi")
        cond_out1 = self.cond_fc(cond_context)
        #print("hi")
        #proc_out1 = self.proc_fc(proc_pool)
        med_out1 = self.med_fc(med_context)
        #print("hi")
        #lab_out1 = self.lab_fc(lab_pool)
        bmi_out1 = self.bmi_fc(bmi_context)
        #print("hi")
        census_out1 = self.census_fc(census_output)
        
        #print(cond_out1.shape)
        #print(proc_out1.shape)
        #print(med_out1.shape)
        #print(bmi_out1.shape)
        #print(census_out1.shape)
        #out1=torch.cat((cond_out1,proc_out1,med_out1,lab_out1,census_out1),1)
        #out1=torch.add(cond_out1, med_out1)
        
        #out1=torch.add(out1, lab_out1)
        out1=torch.add(cond_out1, bmi_out1)
        #print("hi")
        out1=torch.add(out1, med_out1)
        #print("hi")
        #print(out1.shape)
        #print(census_out1.shape)
        out1=torch.add(out1, census_out1)
        #print("hi")
        #print(out1)
        #out1=self.fc(out1)
        sigout1 = self.sig(out1)
        #print("sig out",out1.shape)
        #print(out1[0])
        #print("hi")
        
        return sigout1
        
            

In [ ]:
class CodeAttn(nn.Module):
    def __init__(self,device,embed_size,rnn_size,code_vocab_size,code_seq_len,time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeAttn, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
        if self.bmi_flag:
            self.codeRnn = nn.LSTM(input_size=self.embed_size + 1,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
        else:
            self.codeEmbed=nn.Embedding(self.code_vocab_size,self.embed_size,self.padding_idx)
            self.codeRnn = nn.LSTM(input_size=self.embed_size*2,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
            
        self.code_max = nn.AdaptiveMaxPool1d(1, True)
        self.code_fc=nn.Linear(self.rnn_size, 1, False)
        self.code_drop = nn.Dropout(0.2)
        
        self.timeEmbed = nn.Embedding(self.time_vocab_size, self.embed_size). \
            from_pretrained(embeddings=self._init_posi_embedding(self.time_vocab_size, self.embed_size))

    def forward(self, code,code_lengths):
        #print(conds.shape)
        code_time=code[1]
        code=code[0]
        
        #initialize hidden and cell state
        h_0, c_0 = self.init_hidden()
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        #Embedd all sequences
        #print("embedded")
        if self.bmi_flag:
            code=code.unsqueeze(2)
            #print("code",code.shape)
            #print(self.code_seq_len)
            #code=code.type(torch.LongTensor)
        else:
            code=self.codeEmbed(code)
        
        #code_time=code_time.type(torch.LongTensor)
        #h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)
        code_time=self.timeEmbed(code_time)
        #print("hi")
        #print(code_time.shape)
        code=code.type(torch.FloatTensor)
        code_time=code_time.type(torch.FloatTensor)
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        code=torch.cat((code,code_time),dim=2)
        #print("hi")
        code=self.code_drop(code) 
        #Pack all sequences
        code_pack = torch.nn.utils.rnn.pack_padded_sequence(code, code_lengths, batch_first=True,enforce_sorted=False)
        
        #Run through LSTM
        code_output, (code_h_n, code_c_n)=self.codeRnn(code_pack, (h_0, c_0))
       
        #unpack sequence
        code_output, _ = torch.nn.utils.rnn.pad_packed_sequence(code_output, batch_first=True)
        #print("output",code_output.shape)
        code_softmax=self.code_fc(code_output)
        #print("softmax",code_softmax.shape)
        code_softmax=F.softmax(code_softmax)
        #print("softmax",code_softmax.shape)
        code_softmax=torch.sum(torch.mul(code_output,code_softmax),dim=1)
        #print("softmax",code_softmax.shape)
        #print("========================")
               
        #code_output=F.relu(code_output)
                      
        
        
        return code_softmax
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


In [ ]:
class LSTMMaxStatic(nn.Module):
    def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,bmi_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(LSTMMaxStatic, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.build()
        
    def build(self):
        self.cond=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.cond_vocab_size,self.cond_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.proc=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.proc_vocab_size,self.proc_seq_len,self.time_vocab_size,self.batch_size)
        self.med=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.med_vocab_size,self.med_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.lab=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.lab_vocab_size,self.lab_seq_len,self.time_vocab_size,self.batch_size)
        self.bmi=CodeLSTM(self.device,self.embed_size,self.rnn_size,None,self.bmi_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=True)
        
        
        self.cond_fc=nn.Linear(self.rnn_size, 1, False)
        #self.proc_fc=nn.Linear(self.rnn_size, 1, False)
        self.med_fc=nn.Linear(self.rnn_size, 1, False)
        #self.lab_fc=nn.Linear(self.rnn_size, 1, False)
        self.bmi_fc=nn.Linear(self.rnn_size, 1, False)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, lab_lengths,bmi, bmi_lengths,census,contrib):        
        
        cond_pool,cond_indices =self.cond(conds, cond_lengths)
        #print("cond")
        #proc_pool,proc_indices = self.proc(procs, proc_lengths)      
        med_pool,med_indices = self.med(meds, med_lengths)    
        #print("med")
        #lab_pool,lab_indices = self.lab(labs, lab_lengths)  
        bmi_pool,bmi_indices =self.bmi(bmi, bmi_lengths)
        #print("bmi")
        
        #print("hi")
        cond_out1 = self.cond_fc(cond_pool)
        #proc_out1 = self.proc_fc(proc_pool)
        med_out1 = self.med_fc(med_pool)
        #lab_out1 = self.lab_fc(lab_pool)
        bmi_out1 = self.bmi_fc(bmi_pool)
       
        #print(cond_out1.shape)
        #print(proc_out1.shape)
        #print(med_out1.shape)
        #print(lab_out1.shape)
        #print(census_out1.shape)
        #out1=torch.cat((cond_out1,proc_out1,med_out1,lab_out1,census_out1),1)
        #out1=torch.add(cond_out1, med_out1)
        
        #out1=torch.add(out1, lab_out1)
        out1=torch.add(cond_out1, bmi_out1)
        #print("hi")
        out1=torch.add(out1, med_out1)
        #print("hi")
        #print(out1.shape)
        #print(census_out1.shape)
        
        #print(out1)
        #out1=self.fc(out1)
        sigout1 = self.sig(out1)
        #print("sig out",out1.shape)
        #print(out1[0])
        #print("hi")
        if contrib:
            cond_pool = cond_pool.data.cpu().numpy()
            #proc_pool = proc_pool.data.cpu().numpy()
            med_pool = med_pool.data.cpu().numpy()
            #lab_pool = lab_pool.data.cpu().numpy()
            bmi_pool = bmi_pool.data.cpu().numpy()
            
            assert cond_pool.shape == cond_indices.shape
            cond_fc_weight = self.cond_fc.weight.data.cpu().numpy().reshape(-1)
            #proc_fc_weight = self.proc_fc.weight.data.cpu().numpy().reshape(-1)
            med_fc_weight = self.med_fc.weight.data.cpu().numpy().reshape(-1)
            #lab_fc_weight = self.lab_fc.weight.data.cpu().numpy().reshape(-1)
            bmi_fc_weight = self.bmi_fc.weight.data.cpu().numpy().reshape(-1)
            census_fc_weight = self.census_fc.weight.data.cpu().numpy().reshape(-1)
            #print("logit_weight",fc_weight.shape)
            cond_contri = np.zeros((self.batch_size, self.cond_seq_len))
            #proc_contri = np.zeros((self.batch_size, self.proc_seq_len))
            med_contri = np.zeros((self.batch_size, self.med_seq_len))
            #lab_contri = np.zeros((self.batch_size, self.lab_seq_len))
            bmi_contri = np.zeros((self.batch_size, self.bmi_seq_len))
            
            for i in range(cond_pool.shape[0]):
                for j in range(cond_pool.shape[1]):
                    con = cond_pool[i,j] * cond_fc_weight[j]
                    idx = cond_indices[i, j]
                    cond_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(proc_pool.shape[0]):
#                for j in range(proc_pool.shape[1]):
#                    con = proc_pool[i,j] * proc_fc_weight[j]
#                    idx = proc_indices[i, j]
#                    proc_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
            for i in range(med_pool.shape[0]):
                for j in range(med_pool.shape[1]):
                    con = med_pool[i,j] * med_fc_weight[j]
                    idx = med_indices[i, j]
                    med_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(lab_pool.shape[0]):
#                for j in range(lab_pool.shape[1]):
#                    con = lab_pool[i,j] * lab_fc_weight[j]
#                    idx = lab_indices[i, j]
#                    lab_contri[i,idx] += con
            for i in range(bmi_pool.shape[0]):
                for j in range(bmi_pool.shape[1]):
                    con = bmi_pool[i,j] * bmi_fc_weight[j]
                    idx = bmi_indices[i, j]
                    bmi_contri[i,idx] += con
            #print("contrib",cond_contri.shape)
            
            #print("contrib",cond_contri[0])
            #cond_output = cond_output.data.cpu().numpy()
            #return sigout1,out1, cond_contri,proc_contri,med_contri,lab_contri,census_contri,cond_pool,cond_fc_weight,cond_indices
            return sigout1,out1, cond_contri,bmi_contri,med_contri,cond_pool,cond_fc_weight,cond_indices
        else:
            return sigout1
        
            

In [ ]:
class LSTMMax(nn.Module):
    def __init__(self,device,cond_vocab_size,cond_seq_len,proc_vocab_size,proc_seq_len,med_vocab_size,med_seq_len,lab_vocab_size,lab_seq_len,bmi_vocab_size,bmi_seq_len,time_vocab_size,census_vocab_size,embed_size,rnn_size,batch_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(LSTMMax, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_vocab_size=bmi_vocab_size
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=1
        self.device=device
        self.build()
        
    def build(self):
        self.cond=CodeLSTMAll(self.device,self.embed_size,self.rnn_size,self.cond_vocab_size,self.cond_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        self.proc=CodeLSTMAll(self.device,self.embed_size,self.rnn_size,self.proc_vocab_size,self.proc_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        self.med=CodeLSTMAll(self.device,self.embed_size,self.rnn_size,self.med_vocab_size,self.med_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=False)
        #self.lab=CodeLSTM(self.device,self.embed_size,self.rnn_size,self.lab_vocab_size,self.lab_seq_len,self.time_vocab_size,self.batch_size)
        #self.bmi=CodeLSTMAll(self.device,self.embed_size,self.rnn_size,self.bmi_vocab_size,self.bmi_seq_len,self.time_vocab_size,self.batch_size,bmi_flag=True)
        
        #self.censusEmbed=nn.Embedding(self.census_vocab_size,self.embed_size,self.padding_idx)             
        #self.census_max = nn.AdaptiveMaxPool1d(1, True)        
        
        self.fc=nn.Linear(3*self.rnn_size, 1, False)
        #self.cond_fc=nn.Linear(self.rnn_size, 1, False)
        #self.proc_fc=nn.Linear(self.rnn_size, 1, False)
        #self.med_fc=nn.Linear(self.rnn_size, 1, False)
        #self.lab_fc=nn.Linear(self.rnn_size, 1, False)
        #self.bmi_fc=nn.Linear(self.rnn_size, 1, False)
        #self.census_fc=nn.Linear(self.embed_size, 1, False)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,contrib):        
        
        cond_pool,cond_indices =self.cond(conds, cond_lengths)
        #print("cond_pool",cond_pool.shape)
        #cond_pool,cond_indices,cond_pool_ob,cond_indices_ob =self.cond(conds, cond_lengths)
        #print("cond")
        proc_pool,proc_indices = self.proc(procs, proc_lengths)   
        #print("cond_pool",proc_pool.shape)
        med_pool,med_indices = self.med(meds, med_lengths)  
        #print("cond_pool",med_pool.shape)
        #med_pool,med_indices,med_pool_ob,med_indices_ob =self.med(meds, med_lengths)
        #print("med")
        #lab_pool,lab_indices = self.lab(labs, lab_lengths)  
        #bmi_pool,bmi_indices =self.bmi(bmi, bmi_lengths)
        #print("bmi")
#         census=census.to(self.device)              
#         census=self.censusEmbed(census)     
#         census_output=census.view(census.shape[0],census.shape[2],census.shape[1])
#         census_output=F.relu(census_output)
#         census_pool, census_indices = self.census_max(census_output)               
#         census_pool,census_indices  = torch.squeeze(census_pool), torch.squeeze(census_indices)

        #print("hi")
        #cond_out1 = self.cond_fc(cond_pool)
        #proc_out1 = self.proc_fc(proc_pool)
        #med_out1 = self.med_fc(med_pool)
        #lab_out1 = self.lab_fc(lab_pool)
        #bmi_out1 = self.bmi_fc(bmi_pool)
        #census_out1 = self.census_fc(census_pool)
        
        
        #print("cond_pool_ob",cond_pool_ob.shape)
        #out1=torch.cat((cond_pool,cond_pool_ob),1)
        #out1=cond_pool
        out1=torch.cat((cond_pool,med_pool),1)
        out1=torch.cat((out1,proc_pool),1)
        out1 = self.fc(out1)
        #print("out1",out1.shape)
        #print("out1",out1)
        #print("out1",out1[16])
        #out1=torch.add(cond_out1, med_out1)
        fc_w=self.fc.weight.data
        fc_w=torch.squeeze(fc_w)
#         print("fc_w",fc_w)
        
#         print("Next code_out",cond_pool[10])
#         print("Next code_out_ob",cond_pool_ob[10])
        
        #code_out=torch.mul(cond_pool,fc_w[0:cond_pool.shape[1]])
        #code_out_ob=torch.mul(cond_pool_ob,fc_w[cond_pool.shape[1]:])
        #med_out=torch.mul(med_pool,fc_w[cond_pool.shape[1]:])
        
        #print("Next code_out",cond_pool[16])
        #print("Cond Indices",cond_indices[16])
##         print("Next code_out_ob",code_out_ob[10])
        #print("Next med_out",med_pool[16])
        #print("Med Indices",med_indices[16])
        #print("Next code_out",code_out[16].sum())
##         print("Next code_out_ob",code_out_ob[10].sum())
        #print("Next med_out",med_out[16].sum())
##         print(code_out[0,:])
        
        
        sigout1 = self.sig(out1)
        #print("sig out",sigout1[16])
        #print("sig out",sigout1)
        #print(out1[0])
        #print("hi")
        if contrib:
            cond_pool = cond_pool.data.cpu().numpy()
            #proc_pool = proc_pool.data.cpu().numpy()
            med_pool = med_pool.data.cpu().numpy()
            #lab_pool = lab_pool.data.cpu().numpy()
            bmi_pool = bmi_pool.data.cpu().numpy()
            census_pool = census_pool.data.cpu().numpy()
            #print(cond_pool.shape)
            assert cond_pool.shape == cond_indices.shape
            cond_fc_weight = self.cond_fc.weight.data.cpu().numpy().reshape(-1)
            #proc_fc_weight = self.proc_fc.weight.data.cpu().numpy().reshape(-1)
            med_fc_weight = self.med_fc.weight.data.cpu().numpy().reshape(-1)
            #lab_fc_weight = self.lab_fc.weight.data.cpu().numpy().reshape(-1)
            bmi_fc_weight = self.bmi_fc.weight.data.cpu().numpy().reshape(-1)
            census_fc_weight = self.census_fc.weight.data.cpu().numpy().reshape(-1)
            #print("logit_weight",fc_weight.shape)
            cond_contri = np.zeros((self.batch_size, self.cond_seq_len))
            #proc_contri = np.zeros((self.batch_size, self.proc_seq_len))
            med_contri = np.zeros((self.batch_size, self.med_seq_len))
            #lab_contri = np.zeros((self.batch_size, self.lab_seq_len))
            bmi_contri = np.zeros((self.batch_size, self.bmi_seq_len))
            census_contri = np.zeros((self.batch_size, self.census_vocab_size))
            #print("contrib",cond_contri.shape)
            for i in range(cond_pool.shape[0]):
                for j in range(cond_pool.shape[1]):
                    con = cond_pool[i,j] * cond_fc_weight[j]
                    idx = cond_indices[i, j]
                    cond_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(proc_pool.shape[0]):
#                for j in range(proc_pool.shape[1]):
#                    con = proc_pool[i,j] * proc_fc_weight[j]
#                    idx = proc_indices[i, j]
#                    proc_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
            for i in range(med_pool.shape[0]):
                for j in range(med_pool.shape[1]):
                    con = med_pool[i,j] * med_fc_weight[j]
                    idx = med_indices[i, j]
                    med_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(lab_pool.shape[0]):
#                for j in range(lab_pool.shape[1]):
#                    con = lab_pool[i,j] * lab_fc_weight[j]
#                    idx = lab_indices[i, j]
#                    lab_contri[i,idx] += con
            for i in range(bmi_pool.shape[0]):
                for j in range(bmi_pool.shape[1]):
                    con = bmi_pool[i,j] * bmi_fc_weight[j]
                    idx = bmi_indices[i, j]
                    bmi_contri[i,idx] += con
            #print("contrib",cond_contri.shape)
            for i in range(census_pool.shape[0]):
                for j in range(census_pool.shape[1]):
                    con = census_pool[i,j] * census_fc_weight[j]
                    idx = census_indices[i, j]
                    census_contri[i,idx] += con
            #print("contrib",cond_contri[0])
            #cond_output = cond_output.data.cpu().numpy()
            #return sigout1,out1, cond_contri,proc_contri,med_contri,lab_contri,census_contri,cond_pool,cond_fc_weight,cond_indices
            return sigout1,out1, cond_contri,bmi_contri,census_contri,med_contri,cond_pool,cond_fc_weight,cond_indices
        else:
            return sigout1,out1
        
            

In [1]:
class CodeLSTMAll(nn.Module):
    def __init__(self,device,embed_size,rnn_size,code_vocab_size,code_seq_len,time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeLSTMAll, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
#         if self.bmi_flag:
#             self.codeRnn = nn.LSTM(input_size=self.embed_size,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
#         else:
        self.codeEmbed=nn.Embedding(self.code_vocab_size,self.embed_size,self.padding_idx)
        self.codeRnn = nn.LSTM(input_size=self.embed_size,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)

        
#        self.timeEmbed = nn.Embedding(self.time_vocab_size, self.embed_size). \
#            from_pretrained(embeddings=self._init_posi_embedding(self.time_vocab_size, self.embed_size))
        
        self.code_max = nn.AdaptiveMaxPool1d(1, True)
       
        self.code_drop = nn.Dropout(0.2)
        
    def forward(self, code,code_lengths):
        #print(conds.shape)
        #ob=code[2]
#        code_time=code[1]
#        code=code[0]
        #print()
        #initialize hidden and cell state
        h_0, c_0 = self.init_hidden()
        h_0, c_0, code = h_0.to(self.device), c_0.to(self.device),code.to(self.device)

        #Embedd all sequences
        #print(code.shape)
        #print("embedded")
#         if self.bmi_flag:
#             code=code.unsqueeze(2)
#             #print("code",code.shape)
#             #print(self.code_seq_len)
#             #code=code.type(torch.LongTensor)
#         else:
        code=self.codeEmbed(code)
        
#        code_time=code_time.type(torch.LongTensor)
        h_0, c_0, code = h_0.to(self.device), c_0.to(self.device),code.to(self.device)
#        code_time=self.timeEmbed(code_time)
        #print("hi")
        #print(code.shape)
        code=code.type(torch.FloatTensor)
#        code_time=code_time.type(torch.FloatTensor)
        h_0, c_0, code = h_0.to(self.device), c_0.to(self.device),code.to(self.device)

#        code=torch.cat((code,code_time),dim=2)
        #print("hi")
        #code=self.code_drop(code) 
        #Pack all sequences
        code_pack = torch.nn.utils.rnn.pack_padded_sequence(code, code_lengths, batch_first=True,enforce_sorted=False)
        
        #Run through LSTM
        code_output, (code_h_n, code_c_n)=self.codeRnn(code_pack, (h_0, c_0))
       
        #unpack sequence
        code_output, _ = torch.nn.utils.rnn.pad_packed_sequence(code_output, batch_first=True)
        
               
        
        #code_output=F.relu(code_output)
                      
        #print("output",code_output.shape)
        #print("hi")
        code_output=code_output.view(code_output.shape[0],code_output.shape[2],code_output.shape[1])
        
        #code_output=F.relu(code_output)
#         print("output relu",code_output[99])
        #print("======================")
        #print(code_output.shape)
        #print(code_lengths.shape)
        #print("======================")
        
        code_pool=[]
        code_indices=[]
        
        for i in range(code_output.shape[0]):
            pool, indices = self.code_max(code_output[i:i+1,:,0:code_lengths[i]])
            if i==0:
                code_pool=pool
                code_indices=indices
            else:
                code_pool=torch.cat((code_pool,pool),0)
                code_indices=torch.cat((code_indices,indices),0)
        #print("======================")
        ##print("code_indices",code_indices.shape)
        #print("======================")
        
 
        code_pool = torch.squeeze(code_pool)
        code_indices = torch.squeeze(code_indices)
        #print("code_indices",code_indices.shape)
        
        
        return code_pool,code_indices
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


NameError: name 'nn' is not defined

In [ ]:
class CodeLSTMob(nn.Module):
    def __init__(self,device,embed_size,rnn_size,code_vocab_size,code_seq_len,time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeLSTMob, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
        if self.bmi_flag:
            self.codeRnn = nn.LSTM(input_size=self.embed_size + 1,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
        else:
            self.codeEmbed=nn.Embedding(self.code_vocab_size,self.embed_size,self.padding_idx)
            self.codeRnn = nn.LSTM(input_size=2*self.embed_size,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
            
        
        self.timeEmbed = nn.Embedding(self.time_vocab_size, self.embed_size). \
            from_pretrained(embeddings=self._init_posi_embedding(self.time_vocab_size, self.embed_size))
        
        self.code_max = nn.AdaptiveMaxPool1d(1, True)
        self.code_max_ob = nn.AdaptiveMaxPool1d(1, True)
       
        self.code_drop = nn.Dropout(0.2)
        
    def forward(self, code,code_lengths):
        #print(conds.shape)
        ob=code[2]
        code_time=code[1]
        code=code[0]
        
        #initialize hidden and cell state
        h_0, c_0 = self.init_hidden()
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        #Embedd all sequences
        #print("embedded")
        if self.bmi_flag:
            code=code.unsqueeze(2)
            #print("code",code.shape)
            #print(self.code_seq_len)
            #code=code.type(torch.LongTensor)
        else:
            code=self.codeEmbed(code)
        
        code_time=code_time.type(torch.LongTensor)
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)
        code_time=self.timeEmbed(code_time)
        #print("hi")
#         print(code.shape)
        code=code.type(torch.FloatTensor)
        code_time=code_time.type(torch.FloatTensor)
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        code=torch.cat((code,code_time),dim=2)
        #print("hi")
        #code=self.code_drop(code) 
        #Pack all sequences
        code_pack = torch.nn.utils.rnn.pack_padded_sequence(code, code_lengths, batch_first=True,enforce_sorted=False)
        
        #Run through LSTM
        code_output, (code_h_n, code_c_n)=self.codeRnn(code_pack, (h_0, c_0))
       
        #unpack sequence
        code_output, _ = torch.nn.utils.rnn.pad_packed_sequence(code_output, batch_first=True)
        
               
        
        #code_output=F.relu(code_output)
                      
#         print("output",code_output.shape)
        #print("hi")
        code_output=code_output.view(code_output.shape[0],code_output.shape[2],code_output.shape[1])
        
        code_output=F.relu(code_output)
#         print("output relu",code_output[99])     
#         print(code_output.shape)
        ob=ob.unsqueeze(1).repeat(1,self.rnn_size,1)
#         print("ob",ob.shape)
        ob=ob[:,:,0:code_output.shape[2]]
#         print("ob",ob.shape)
        ob=ob.type(torch.FloatTensor)
        #print("ob",ob[2,:])
        ob=ob.to(self.device)
        #print((code_output)[2,:,:])
        #print((code_output*ob)[2,:,:])
        code_pool_ob, code_indices_ob = self.code_max_ob(code_output*ob)
        ob=(~(ob.int().bool())).int()
#         print("ob",ob[2,:])
        ob=ob.type(torch.FloatTensor)
        ob=ob.to(self.device)
        #print((code_output*ob)[2,:,:])
        code_pool, code_indices = self.code_max(code_output*ob)
        #print("code_pool",code_pool.shape)
        
 
        code_pool = torch.squeeze(code_pool)
        code_indices = torch.squeeze(code_indices)
        code_pool_ob = torch.squeeze(code_pool_ob)
        code_indices_ob = torch.squeeze(code_indices_ob)
#         print("code_indices",code_indices.shape)
#         print("code_indices_ob",code_indices_ob.shape)
        #print("code_pool",code_pool.shape)
        #print(code_pool_ob[2,:])
        #print(code_pool[2,:])
        
        
        
        return code_pool,code_indices,code_pool_ob,code_indices_ob
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


In [ ]:
class CodeLSTM(nn.Module):
    def __init__(self,device,embed_size,rnn_size,code_vocab_size,code_seq_len,time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeLSTM, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
        if self.bmi_flag:
            self.codeRnn = nn.LSTM(input_size=self.embed_size + 1,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
        else:
            self.codeEmbed=nn.Embedding(self.code_vocab_size,self.embed_size,self.padding_idx)
            self.codeRnn = nn.LSTM(input_size=self.embed_size*2,hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
            
        self.code_max = nn.AdaptiveMaxPool1d(1, True)
        self.code_fc=nn.Linear(self.rnn_size, 1, False)
        self.code_drop = nn.Dropout(0.2)
        
        self.timeEmbed = nn.Embedding(self.time_vocab_size, self.embed_size). \
            from_pretrained(embeddings=self._init_posi_embedding(self.time_vocab_size, self.embed_size))

    def forward(self, code,code_lengths):
        #print(conds.shape)
        code_time=code[1]
        code=code[0]
        
        #initialize hidden and cell state
        h_0, c_0 = self.init_hidden()
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        #Embedd all sequences
        #print("embedded")
        if self.bmi_flag:
            code=code.unsqueeze(2)
            #print("code",code.shape)
            #print(self.code_seq_len)
            #code=code.type(torch.LongTensor)
        else:
            code=self.codeEmbed(code)
        
        #code_time=code_time.type(torch.LongTensor)
        #h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)
        code_time=self.timeEmbed(code_time)
        #print("hi")
        #print(code_time.shape)
        code=code.type(torch.FloatTensor)
        code_time=code_time.type(torch.FloatTensor)
        h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)

        code=torch.cat((code,code_time),dim=2)
        #print("hi")
        code=self.code_drop(code) 
        #Pack all sequences
        code_pack = torch.nn.utils.rnn.pack_padded_sequence(code, code_lengths, batch_first=True,enforce_sorted=False)
        
        #Run through LSTM
        code_output, (code_h_n, code_c_n)=self.codeRnn(code_pack, (h_0, c_0))
       
        #unpack sequence
        code_output, _ = torch.nn.utils.rnn.pad_packed_sequence(code_output, batch_first=True)
        
               
        
        #code_output=F.relu(code_output)
                      
        #print("output",cond_output.shape)
        #print("hi")
        code_output=code_output.view(code_output.shape[0],code_output.shape[2],code_output.shape[1])
        
        code_pool, code_indices = self.code_max(code_output)
        #print("indices",cond_indices.shape)
 
        code_pool = torch.squeeze(code_pool)
        code_indices = torch.squeeze(code_indices)
        
        return code_pool,code_indices
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


In [ ]:
class LSTMMaxNew(nn.Module):
    def __init__(self,device,
                 cond_vocab_size,cond_seq_len,proc_vocab_size,
                 proc_seq_len,med_vocab_size,med_seq_len,
                 lab_vocab_size,lab_seq_len,
                 bmi_seq_len,time_vocab_size,
                 census_vocab_size,
                 embed_size,rnn_size,
                 batch_size,filter_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(LSTMMaxNew, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.ndepth=2
        self.nbreadth=4
        self.filter_size=filter_size
        #self.meds={}
        self.build()
        
    def build(self):
        self.cond=nn.ModuleList()
#         self.med=nn.ModuleList()
        
        #if self.cond_seq_len%self.filter_size>0:
         ##   pad=int(self.filter_size-(self.cond_seq_len%self.filter_size))
         #   self.cond_seq_len=self.cond_seq_len+pad
        nfilter= int(self.cond_seq_len/self.filter_size)
        self.cond.append(nn.ModuleList([
                CodeLSTMNew(self.device,
                          self.embed_size,self.rnn_size,
                          self.cond_vocab_size,self.cond_seq_len,
                          self.time_vocab_size,
                          self.batch_size,bmi_flag=False) for k in range(nfilter)]))

#         self.med.append(nn.ModuleList([
#             CodeLSTMNew(self.device,
#                       self.embed_size,self.rnn_size,
#                       self.cond_vocab_size,self.cond_seq_len,
#                       self.time_vocab_size,
#                       self.batch_size,bmi_flag=False) for k in range(4)]))
        nfilter= int(nfilter/self.filter_size)
        while nfilter>=4:           
            self.cond.append(nn.ModuleList([
                CodeLSTMNew2(self.device,
                          self.embed_size,self.rnn_size,
                          self.cond_vocab_size,self.cond_seq_len,
                          self.time_vocab_size,
                          self.batch_size,bmi_flag=False) for k in range(nfilter)]))
            nfilter= (nfilter/self.filter_size)
        #height=len(self.cond)
        #root=len(self.cond[height-1])
        
        self.cond.append(nn.ModuleList([
                CodeLSTMNew2(self.device,
                          self.embed_size,self.rnn_size,
                          self.cond_vocab_size,self.cond_seq_len,
                          self.time_vocab_size,
                          self.batch_size,bmi_flag=False)]))
#             self.med.append(nn.ModuleList([
#                 CodeLSTMNew2(self.device,
#                           self.embed_size,self.rnn_size,
#                           self.cond_vocab_size,self.cond_seq_len,
#                           self.time_vocab_size,
#                           self.batch_size,bmi_flag=False) for k in range(j)]))
        
        self.sig = nn.Sigmoid()
        self.shared_fc=nn.Linear(2*self.rnn_size, 1, False)
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, 
                lab_lengths,bmi, bmi_lengths,census,contrib):        
        for i in range(len(self.cond)):
            #print(i)
            #print(cond_lengths)
            new_code_lengths=np.zeros(self.batch_size)
            for j in range(len(self.cond[i])):
                #print("Hi")
                if(i==0):
                    #print("in if")
                    #print(j)
                    start=j*self.filter_size
                    end=(j+1)*self.filter_size
                    #print(start)
                    #print(end)
                    
                    #print(self.cond[i][j])
                    #print(conds[:,:,start:end].shape)
                    #print(cond_lengths)
                    cond_lengths_next=[x - self.filter_size for x in cond_lengths]
                    #print(cond_lengths_next)
                    cond_lengths_next=np.asarray(cond_lengths_next)
                    cond_lengths_next[cond_lengths_next<0]=0
                    cond_lengths_next=list(cond_lengths_next)
                    cond_lengths_current=[a - b for a, b in zip(cond_lengths, cond_lengths_next)]
                    cond_lengths_current=np.asarray(cond_lengths_current)
                    cond_lengths_current[cond_lengths_current==0]=1
                    cond_lengths_current=list(cond_lengths_current)
                    #print(cond_lengths_current)
                    cond_lengths=cond_lengths_next
#                     if torch.max(conds[:,:,start:end]):
#                    else:
#                     pad=(len(self.cond[i])-j)*2
#                     padding=torch.zeros(cond_max.shape[0],pad,cond_max.shape[2])
#                     padding=padding.to(self.device)
#                     cond_max=torch.cat((cond_max,padding),2)
                    
                    cond_pool,cond_indices,fc_cond, cond_out,cond_pool_ob,cond_indices_ob,fc_cond_ob, cond_out_ob,new_code_lengths = self.cond[i][j](conds[:,:,start:end],cond_lengths_current,new_code_lengths,j)
                elif(i==len(self.cond)-1): 
                    cond_pool,cond_indices,fc_cond, cond_out,cond_pool_ob,cond_indices_ob,fc_cond_ob, cond_out_ob,new_code_lengths = self.cond[i][j](conds,cond_lengths_current,new_code_lengths,j)
                else:
                    #print("in i else")
                    #print(j)
                    start=j*self.filter_size
                    end=(j+1)*self.filter_size
                    cond_pool,cond_indices,fc_cond, cond_out,cond_pool_ob,cond_indices_ob,fc_cond_ob, cond_out_ob,new_code_lengths = self.cond[i][j](conds[:,start:end,:],cond_lengths_current,new_code_lengths,j)
                if(j==0):
                    cond_out=torch.cat((cond_out,cond_out_ob),dim=1)
                    #print("max",cond_out.shape)
                    cond_max=cond_out.unsqueeze(1)
                    #cond_max_ob=cond_out_ob.unsqueeze(1)
                    #print(cond_max.shape)
                    #print(cond_max_ob.shape)
                    #cond_max=torch.cat((cond_max, cond_max_ob), 1)
                    #cond_max=cond_out
                    #print("conds max",cond_max.shape)
                else:
                    cond_out=torch.cat((cond_out,cond_out_ob),dim=1)
                    cond_max=torch.cat((cond_max,cond_out.unsqueeze(1)),dim=1)
                    #cond_max=torch.cat((cond_max, torch.cat((cond_out.unsqueeze(1),cond_out_ob.unsqueeze(1)),1)), 1)
                    #print("conds max",cond_max.shape)
            conds=cond_max
            code_lengths=new_code_lengths
            print("code_lengths",code_lengths[99])
            #meds=med_max
            #print("conds max",conds.shape)
        #print('outside for')
        conds=conds.squeeze()
        #print(conds.shape)
        conds=torch.reshape(conds,(conds.shape[0],-1))
        #meds=meds.squeeze()
        #print("conds max",conds.shape)
        #allfeat=torch.cat((conds,meds),1)
        #print(allfeat.shape)
        fc_out=self.shared_fc(conds)
        #print("fc_out",fc_out)
        sigout=self.sig(fc_out)
        #print(sigout)
        if contrib:
            cond_pool = cond_pool.data.cpu().numpy()
            #proc_pool = proc_pool.data.cpu().numpy()
            med_pool = med_pool.data.cpu().numpy()
            #lab_pool = lab_pool.data.cpu().numpy()
            bmi_pool = bmi_pool.data.cpu().numpy()
            census_pool = census_pool.data.cpu().numpy()
            #print(cond_pool.shape)
            assert cond_pool.shape == cond_indices.shape
            cond_fc_weight = self.cond_fc.weight.data.cpu().numpy().reshape(-1)
            #proc_fc_weight = self.proc_fc.weight.data.cpu().numpy().reshape(-1)
            med_fc_weight = self.med_fc.weight.data.cpu().numpy().reshape(-1)
            #lab_fc_weight = self.lab_fc.weight.data.cpu().numpy().reshape(-1)
            bmi_fc_weight = self.bmi_fc.weight.data.cpu().numpy().reshape(-1)
            census_fc_weight = self.census_fc.weight.data.cpu().numpy().reshape(-1)
            #print("logit_weight",fc_weight.shape)
            cond_contri = np.zeros((self.batch_size, self.cond_seq_len))
            #proc_contri = np.zeros((self.batch_size, self.proc_seq_len))
            med_contri = np.zeros((self.batch_size, self.med_seq_len))
            #lab_contri = np.zeros((self.batch_size, self.lab_seq_len))
            bmi_contri = np.zeros((self.batch_size, self.bmi_seq_len))
            census_contri = np.zeros((self.batch_size, self.census_vocab_size))
            #print("contrib",cond_contri.shape)
            for i in range(cond_pool.shape[0]):
                for j in range(cond_pool.shape[1]):
                    con = cond_pool[i,j] * cond_fc_weight[j]
                    idx = cond_indices[i, j]
                    cond_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(proc_pool.shape[0]):
#                for j in range(proc_pool.shape[1]):
#                    con = proc_pool[i,j] * proc_fc_weight[j]
#                    idx = proc_indices[i, j]
#                    proc_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
            for i in range(med_pool.shape[0]):
                for j in range(med_pool.shape[1]):
                    con = med_pool[i,j] * med_fc_weight[j]
                    idx = med_indices[i, j]
                    med_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(lab_pool.shape[0]):
#                for j in range(lab_pool.shape[1]):
#                    con = lab_pool[i,j] * lab_fc_weight[j]
#                    idx = lab_indices[i, j]
#                    lab_contri[i,idx] += con
            for i in range(bmi_pool.shape[0]):
                for j in range(bmi_pool.shape[1]):
                    con = bmi_pool[i,j] * bmi_fc_weight[j]
                    idx = bmi_indices[i, j]
                    bmi_contri[i,idx] += con
            #print("contrib",cond_contri.shape)
            for i in range(census_pool.shape[0]):
                for j in range(census_pool.shape[1]):
                    con = census_pool[i,j] * census_fc_weight[j]
                    idx = census_indices[i, j]
                    census_contri[i,idx] += con
            #print("contrib",cond_contri[0])
            #cond_output = cond_output.data.cpu().numpy()
            #return sigout1,out1, cond_contri,proc_contri,med_contri,lab_contri,census_contri,cond_pool,cond_fc_weight,cond_indices
            return sigout,out1, cond_contri,bmi_contri,census_contri,med_contri,cond_pool,cond_fc_weight,cond_indices
        else:
            return sigout
        
            

In [ ]:
class CodeLSTMNew(nn.Module):
    def __init__(self,device,embed_size,rnn_size,code_vocab_size,code_seq_len,
                 time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeLSTMNew, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
        if self.bmi_flag:
            self.codeRnn = nn.LSTM(input_size=self.embed_size + 1,
                                   hidden_size=self.rnn_size,num_layers = 1,batch_first=True)
        else:
            self.codeEmbed=nn.Embedding(self.code_vocab_size,self.embed_size,self.padding_idx)
            self.codeRnn = nn.LSTM(input_size=self.embed_size,hidden_size=self.rnn_size,
                                   num_layers = 1,batch_first=True)
            
        self.code_max = nn.AdaptiveMaxPool1d(1, True)
        self.code_max_ob = nn.AdaptiveMaxPool1d(1, True)
        self.code_fc=nn.Linear(self.rnn_size, 1, False)
        self.code_drop = nn.Dropout(0.2)
        
        self.code_fc=nn.Linear(self.rnn_size, 1, False)
        self.code_fc_ob=nn.Linear(self.rnn_size, 1, False)

    def forward(self, code,code_lengths,new_code_lengths,j):
        #print("forward of codelstmnew")
        ob=code[2]
        code_time=code[1]
        code=code[0]
        
        for j_len in range(code.shape[0]):
            if(code[j_len].sum()==0):
                new_code_lengths[j_len]=j
        print("cond",code[99])
        #print("ob",ob.shape)
        #print(code.shape)
        #print(len(code_lengths))
        #initialize hidden and cell state
        h_0, c_0 = self.init_hidden()
        h_0, c_0, code_time, code,ob = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device),ob.to(self.device)
        #print("code",code[0:10,:])
        #Embedd all sequences
        #print("embedded")
        if self.bmi_flag:
            code=code.unsqueeze(2)
            #print("code",code.shape)
            #print(self.code_seq_len)
            #code=code.type(torch.LongTensor)
        else:
            code=self.codeEmbed(code)
        
        #code_time=code_time.type(torch.LongTensor)
        #h_0, c_0, code_time, code = h_0.to(self.device), c_0.to(self.device),code_time.to(self.device),code.to(self.device)
        
        #print("hi")
        #print("embed code",code[99])
        #print(code_lengths)
        code=code.type(torch.FloatTensor)
        h_0, c_0, code = h_0.to(self.device), c_0.to(self.device),code.to(self.device)

        #code=self.code_drop(code) 
        #Pack all sequences
        code_pack = torch.nn.utils.rnn.pack_padded_sequence(code, code_lengths, batch_first=True,enforce_sorted=False)
        
        #Run through LSTM
        code_output, (code_h_n, code_c_n)=self.codeRnn(code_pack, (h_0, c_0))
        
        #unpack sequence
        code_output, _ = torch.nn.utils.rnn.pad_packed_sequence(code_output, batch_first=True)
        #print("output rnn",code_output[99])
               
        
        code_output=F.relu(code_output)
        print("output relu",code_output[99])     
        #print("output relu",code_output[0:10,:,:])
        #print("hi")
        code_output=code_output.view(code_output.shape[0],code_output.shape[2],code_output.shape[1])
        #print("output reshaped",code_output.shape)
        self.filter_size=10
        if(code_output.shape[2]<self.filter_size):
            pad=self.filter_size-code_output.shape[2]
            padding=torch.zeros(code_output.shape[0],code_output.shape[1],pad)
            padding=padding.to(self.device)
            code_output=torch.cat((code_output,padding),2)
            #print("output reshaped",code_output.shape)
        #print("ob",ob[2,:])
        ob=ob.unsqueeze(1).repeat(1,self.rnn_size,1)
        #print("ob",ob.shape)
        ob=ob.type(torch.FloatTensor)
        #print("ob",ob[2,:])
        ob=ob.to(self.device)
        #print((code_output)[2,:,:])
        #print((code_output*ob)[2,:,:])
        code_pool_ob, code_indices_ob = self.code_max_ob(code_output*ob)
        ob=(~(ob.int().bool())).int()
        #print("ob",ob[2,:])
        ob=ob.type(torch.FloatTensor)
        ob=ob.to(self.device)
        #print((code_output*ob)[2,:,:])
        code_pool, code_indices = self.code_max(code_output*ob)
        #print("code_pool",code_pool.shape)
        
 
        code_pool = torch.squeeze(code_pool)
        code_indices = torch.squeeze(code_indices)
        code_pool_ob = torch.squeeze(code_pool_ob)
        code_indices_ob = torch.squeeze(code_indices_ob)
        print("code_indices",code_indices[99])
        print("code_indices_ob",code_indices_ob[99])
        #print("code_pool",code_pool.shape)
        #print(code_pool_ob[2,:])
        #print(code_pool[2,:])
        
        code_out = self.code_fc(code_pool)
        code_out_ob = self.code_fc_ob(code_pool_ob)
        print("code_out",code_out[99])
        print("code_out_ob",code_out_ob[99])
        
        fc_w=self.code_fc.weight.data
        fc_w=torch.squeeze(fc_w)
        
        fc_w_ob=self.code_fc_ob.weight.data
        fc_w_ob=torch.squeeze(fc_w_ob)
        #print("fc_w",fc_w.shape)
        #print(fc_w)
        
        code_out=torch.mul(code_pool,self.code_fc.weight.data)
        code_out_ob=torch.mul(code_pool_ob,self.code_fc_ob.weight.data)
        print("Next code_out",code_out[99])
        print("Next code_out_ob",code_out_ob[99])
        print("Next code_out",code_out[99].sum())
        print("Next code_out_ob",code_out_ob[99].sum())
        print(code_out[0,:])
        return code_pool,code_indices,fc_w, code_out,code_pool_ob,code_indices_ob,fc_w_ob,code_out_ob, new_code_lengths
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


In [ ]:
class CodeLSTMNew2(nn.Module):
    def __init__(self,device,embed_size,rnn_size,
                 code_vocab_size,code_seq_len,
                 time_vocab_size,batch_size,bmi_flag=False):             
        super(CodeLSTMNew2, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.code_vocab_size=code_vocab_size
        self.code_seq_len=code_seq_len
        self.batch_size=batch_size
        self.padding_idx = 0
        self.device=device
        self.time_vocab_size=time_vocab_size
        self.bmi_flag=bmi_flag
        self.build()
    
    def build(self):
        self.codeRnn = nn.LSTM(input_size=2*self.rnn_size,hidden_size=self.rnn_size,
                               num_layers = 1,batch_first=True)
            
        self.code_max = nn.AdaptiveMaxPool1d(1, True)
        self.code_max_ob = nn.AdaptiveMaxPool1d(1, True)
        self.code_fc=nn.Linear(self.rnn_size, 1, False)
        self.code_drop = nn.Dropout(0.2)
        
        self.code_fc=nn.Linear(self.rnn_size, 1, False)
        self.code_fc_ob=nn.Linear(self.rnn_size, 1, False)

    def forward(self, code,code_lengths,new_code_lengths,j):        
        #print("in seond layer")
        #initialize hidden and cell state
        for j_len in range(code.shape[0]):
            if(code[j_len].sum()==0):
                new_code_lengths[j_len]=j
        h_0, c_0 = self.init_hidden()
        h_0, c_0,  code = h_0.to(self.device), c_0.to(self.device),code.to(self.device)

        #print(code_time.shape)
        code=code.type(torch.FloatTensor)
        h_0, c_0, code = h_0.to(self.device), c_0.to(self.device),code.to(self.device)

        #Run through LSTM
        #print(code.shape)
        code_pack = torch.nn.utils.rnn.pack_padded_sequence(code, code_lengths, batch_first=True,enforce_sorted=False)
        
        #Run through LSTM
        code_output, (code_h_n, code_c_n)=self.codeRnn(code_pack, (h_0, c_0))
        
        #unpack sequence
        code_output, _ = torch.nn.utils.rnn.pad_packed_sequence(code_output, batch_first=True)
        #print("rnn",code_output.shape)
        code_output=torch.reshape(code_output,(code_output.shape[0],code_output.shape[2],
                                               code_output.shape[1]))
        #print("reshaped",code_output.shape)
        ob=torch.zeros(code_output.shape[0],code_output.shape[1],code_output.shape[2])
        ob=ob.type(torch.FloatTensor)
        ob=ob.to(self.device)
        for k in range(code_output.shape[2]):
            if k%2==1:
                ob[:,:,k]=1
        #print("ob",ob[0,0,:])
        code_pool_ob, code_indices_ob = self.code_max(code_output*ob)
        
        ob=(~(ob.int().bool())).int()
        ob=ob.type(torch.FloatTensor)
        ob=ob.to(self.device)
        code_pool, code_indices = self.code_max(code_output*ob)
        #print("indices",cond_indices.shape)
 
        code_pool = torch.squeeze(code_pool)
        code_indices = torch.squeeze(code_indices)
        code_pool_ob = torch.squeeze(code_pool_ob)
        code_indices_ob = torch.squeeze(code_indices_ob)
        #print("code_pool",code_pool.shape)
        #print(code_pool[0,:])
        code_out = self.code_fc(code_pool)
        code_out_ob = self.code_fc_ob(code_pool_ob)
        #print("code_out",code_out.shape)
        
        fc_w=self.code_fc.weight.data
        fc_w=torch.squeeze(fc_w)
        
        fc_w_ob=self.code_fc_ob.weight.data
        fc_w_ob=torch.squeeze(fc_w_ob)
        #print("fc_w",fc_w.shape)
        #print(fc_w)
        
        code_out=torch.mul(code_pool,self.code_fc.weight.data)
        code_out_ob=torch.mul(code_pool_ob,self.code_fc_ob.weight.data)
        #print("code_out",code_out.shape)
        #print(code_out[0,:])
        return code_pool,code_indices,fc_w, code_out,code_pool_ob,code_indices_ob,fc_w_ob, code_out_ob,new_code_lengths
    
    
    
    def init_hidden(self):
        # initialize the hidden state and the cell state to zeros
        h=torch.zeros(1,self.batch_size, self.rnn_size)
        c=torch.zeros(1,self.batch_size, self.rnn_size)

#         if self.hparams.on_gpu:
#             hidden_a = hidden_a.cuda()
#             hidden_b = hidden_b.cuda()

        h = Variable(h)
        c = Variable(c)

        return (h, c)    
    
    def _init_posi_embedding(self, time_vocab_size, embed_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / embed_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / embed_size)))

        # initialize position embedding table
        lookup_table = np.zeros((time_vocab_size, embed_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(0, embed_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(time_vocab_size):
            for idx in np.arange(1, embed_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)


In [ ]:
class LSTMMaxFlat(nn.Module):
    def __init__(self,device,
                 cond_vocab_size,cond_seq_len,proc_vocab_size,
                 proc_seq_len,med_vocab_size,med_seq_len,
                 lab_vocab_size,lab_seq_len,
                 bmi_seq_len,time_vocab_size,
                 census_vocab_size,
                 embed_size,rnn_size,
                 batch_size,filter_size): #proc_vocab_size,med_vocab_size,lab_vocab_size
        super(LSTMMaxNew, self).__init__()
        self.embed_size=embed_size
        self.rnn_size=rnn_size
        self.cond_vocab_size=cond_vocab_size
        self.cond_seq_len=cond_seq_len
        self.proc_vocab_size=proc_vocab_size
        self.proc_seq_len=proc_seq_len
        self.med_vocab_size=med_vocab_size
        self.med_seq_len=med_seq_len
        self.lab_vocab_size=lab_vocab_size
        self.lab_seq_len=lab_seq_len
        self.bmi_seq_len=bmi_seq_len
        self.time_vocab_size=time_vocab_size
        self.census_vocab_size=census_vocab_size
        self.batch_size=batch_size
        self.padding_idx = 0
        self.modalities=5
        self.device=device
        self.ndepth=2
        self.nbreadth=4
        self.filter_size=filter_size
        #self.meds={}
        self.build()
        
    def build(self):
        self.cond=nn.ModuleList()
#         self.med=nn.ModuleList()
        
        #if self.cond_seq_len%self.filter_size>0:
         ##   pad=int(self.filter_size-(self.cond_seq_len%self.filter_size))
         #   self.cond_seq_len=self.cond_seq_len+pad
        nfilter= int(self.cond_seq_len/self.filter_size)
        self.cond.append(nn.ModuleList([
                CodeLSTMNew(self.device,
                          self.embed_size,self.rnn_size,
                          self.cond_vocab_size,self.cond_seq_len,
                          self.time_vocab_size,
                          self.batch_size,bmi_flag=False) for k in range(nfilter)]))

#         self.med.append(nn.ModuleList([
#             CodeLSTMNew(self.device,
#                       self.embed_size,self.rnn_size,
#                       self.cond_vocab_size,self.cond_seq_len,
#                       self.time_vocab_size,
#                       self.batch_size,bmi_flag=False) for k in range(4)]))
        nfilter= int(nfilter/self.filter_size)
        while nfilter>=4:           
            self.cond.append(nn.ModuleList([
                CodeLSTMNew2(self.device,
                          self.embed_size,self.rnn_size,
                          self.cond_vocab_size,self.cond_seq_len,
                          self.time_vocab_size,
                          self.batch_size,bmi_flag=False) for k in range(nfilter)]))
            nfilter= (nfilter/self.filter_size)
        #height=len(self.cond)
        #root=len(self.cond[height-1])
        
        self.cond.append(nn.ModuleList([
                CodeLSTMNew2(self.device,
                          self.embed_size,self.rnn_size,
                          self.cond_vocab_size,self.cond_seq_len,
                          self.time_vocab_size,
                          self.batch_size,bmi_flag=False)]))
#             self.med.append(nn.ModuleList([
#                 CodeLSTMNew2(self.device,
#                           self.embed_size,self.rnn_size,
#                           self.cond_vocab_size,self.cond_seq_len,
#                           self.time_vocab_size,
#                           self.batch_size,bmi_flag=False) for k in range(j)]))
        
        self.sig = nn.Sigmoid()
        self.shared_fc=nn.Linear(2*self.rnn_size, 1, False)
        
    def forward(self, conds, cond_lengths,procs, proc_lengths,meds, med_lengths,labs, 
                lab_lengths,bmi, bmi_lengths,census,contrib):        
        for i in range(len(self.cond)):
            #print(i)
            #print(cond_lengths)
            new_code_lengths=np.zeros(self.batch_size)
            for j in range(len(self.cond[i])):
                #print("Hi")
                if(i==0):
                    #print("in if")
                    #print(j)
                    start=j*self.filter_size
                    end=(j+1)*self.filter_size
                    #print(start)
                    #print(end)
                    
                    #print(self.cond[i][j])
                    #print(conds[:,:,start:end].shape)
                    #print(cond_lengths)
                    cond_lengths_next=[x - self.filter_size for x in cond_lengths]
                    #print(cond_lengths_next)
                    cond_lengths_next=np.asarray(cond_lengths_next)
                    cond_lengths_next[cond_lengths_next<0]=0
                    cond_lengths_next=list(cond_lengths_next)
                    cond_lengths_current=[a - b for a, b in zip(cond_lengths, cond_lengths_next)]
                    cond_lengths_current=np.asarray(cond_lengths_current)
                    cond_lengths_current[cond_lengths_current==0]=1
                    cond_lengths_current=list(cond_lengths_current)
                    #print(cond_lengths_current)
                    cond_lengths=cond_lengths_next
#                     if torch.max(conds[:,:,start:end]):
#                    else:
#                     pad=(len(self.cond[i])-j)*2
#                     padding=torch.zeros(cond_max.shape[0],pad,cond_max.shape[2])
#                     padding=padding.to(self.device)
#                     cond_max=torch.cat((cond_max,padding),2)
                    
                    cond_pool,cond_indices,fc_cond, cond_out,cond_pool_ob,cond_indices_ob,fc_cond_ob, cond_out_ob,new_code_lengths = self.cond[i][j](conds[:,:,start:end],cond_lengths_current,new_code_lengths,j)
                elif(i==len(self.cond)-1): 
                    cond_pool,cond_indices,fc_cond, cond_out,cond_pool_ob,cond_indices_ob,fc_cond_ob, cond_out_ob,new_code_lengths = self.cond[i][j](conds,cond_lengths_current,new_code_lengths,j)
                else:
                    #print("in i else")
                    #print(j)
                    start=j*self.filter_size
                    end=(j+1)*self.filter_size
                    cond_pool,cond_indices,fc_cond, cond_out,cond_pool_ob,cond_indices_ob,fc_cond_ob, cond_out_ob,new_code_lengths = self.cond[i][j](conds[:,start:end,:],cond_lengths_current,new_code_lengths,j)
                if(j==0):
                    cond_out=torch.cat((cond_out,cond_out_ob),dim=1)
                    #print("max",cond_out.shape)
                    cond_max=cond_out.unsqueeze(1)
                    #cond_max_ob=cond_out_ob.unsqueeze(1)
                    #print(cond_max.shape)
                    #print(cond_max_ob.shape)
                    #cond_max=torch.cat((cond_max, cond_max_ob), 1)
                    #cond_max=cond_out
                    #print("conds max",cond_max.shape)
                else:
                    cond_out=torch.cat((cond_out,cond_out_ob),dim=1)
                    cond_max=torch.cat((cond_max,cond_out.unsqueeze(1)),dim=1)
                    #cond_max=torch.cat((cond_max, torch.cat((cond_out.unsqueeze(1),cond_out_ob.unsqueeze(1)),1)), 1)
                    #print("conds max",cond_max.shape)
            conds=cond_max
            code_lengths=new_code_lengths
            print("code_lengths",code_lengths[99])
            #meds=med_max
            #print("conds max",conds.shape)
        #print('outside for')
        conds=conds.squeeze()
        #print(conds.shape)
        conds=torch.reshape(conds,(conds.shape[0],-1))
        #meds=meds.squeeze()
        #print("conds max",conds.shape)
        #allfeat=torch.cat((conds,meds),1)
        #print(allfeat.shape)
        fc_out=self.shared_fc(conds)
        #print("fc_out",fc_out)
        sigout=self.sig(fc_out)
        #print(sigout)
        if contrib:
            cond_pool = cond_pool.data.cpu().numpy()
            #proc_pool = proc_pool.data.cpu().numpy()
            med_pool = med_pool.data.cpu().numpy()
            #lab_pool = lab_pool.data.cpu().numpy()
            bmi_pool = bmi_pool.data.cpu().numpy()
            census_pool = census_pool.data.cpu().numpy()
            #print(cond_pool.shape)
            assert cond_pool.shape == cond_indices.shape
            cond_fc_weight = self.cond_fc.weight.data.cpu().numpy().reshape(-1)
            #proc_fc_weight = self.proc_fc.weight.data.cpu().numpy().reshape(-1)
            med_fc_weight = self.med_fc.weight.data.cpu().numpy().reshape(-1)
            #lab_fc_weight = self.lab_fc.weight.data.cpu().numpy().reshape(-1)
            bmi_fc_weight = self.bmi_fc.weight.data.cpu().numpy().reshape(-1)
            census_fc_weight = self.census_fc.weight.data.cpu().numpy().reshape(-1)
            #print("logit_weight",fc_weight.shape)
            cond_contri = np.zeros((self.batch_size, self.cond_seq_len))
            #proc_contri = np.zeros((self.batch_size, self.proc_seq_len))
            med_contri = np.zeros((self.batch_size, self.med_seq_len))
            #lab_contri = np.zeros((self.batch_size, self.lab_seq_len))
            bmi_contri = np.zeros((self.batch_size, self.bmi_seq_len))
            census_contri = np.zeros((self.batch_size, self.census_vocab_size))
            #print("contrib",cond_contri.shape)
            for i in range(cond_pool.shape[0]):
                for j in range(cond_pool.shape[1]):
                    con = cond_pool[i,j] * cond_fc_weight[j]
                    idx = cond_indices[i, j]
                    cond_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(proc_pool.shape[0]):
#                for j in range(proc_pool.shape[1]):
#                    con = proc_pool[i,j] * proc_fc_weight[j]
#                    idx = proc_indices[i, j]
#                    proc_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
            for i in range(med_pool.shape[0]):
                for j in range(med_pool.shape[1]):
                    con = med_pool[i,j] * med_fc_weight[j]
                    idx = med_indices[i, j]
                    med_contri[i,idx] += con
            #print("contrib",cond_contri.shape)        
#            for i in range(lab_pool.shape[0]):
#                for j in range(lab_pool.shape[1]):
#                    con = lab_pool[i,j] * lab_fc_weight[j]
#                    idx = lab_indices[i, j]
#                    lab_contri[i,idx] += con
            for i in range(bmi_pool.shape[0]):
                for j in range(bmi_pool.shape[1]):
                    con = bmi_pool[i,j] * bmi_fc_weight[j]
                    idx = bmi_indices[i, j]
                    bmi_contri[i,idx] += con
            #print("contrib",cond_contri.shape)
            for i in range(census_pool.shape[0]):
                for j in range(census_pool.shape[1]):
                    con = census_pool[i,j] * census_fc_weight[j]
                    idx = census_indices[i, j]
                    census_contri[i,idx] += con
            #print("contrib",cond_contri[0])
            #cond_output = cond_output.data.cpu().numpy()
            #return sigout1,out1, cond_contri,proc_contri,med_contri,lab_contri,census_contri,cond_pool,cond_fc_weight,cond_indices
            return sigout,out1, cond_contri,bmi_contri,census_contri,med_contri,cond_pool,cond_fc_weight,cond_indices
        else:
            return sigout

In [ ]:
# class TimeEncoder(nn.Module):
#     def __init__(self, batch_size):
#         super(TimeEncoder, self).__init__()
#         self.batch_size = batch_size
#         self.selection_layer = torch.nn.Linear(1, 64)
#         self.relu = nn.ReLU()
#         self.tanh = nn.Tanh()
#         self.weight_layer = torch.nn.Linear(64, 64)

#     def forward(self, seq_time_step, final_queries, options, mask):
#         if options['use_gpu']:
#             seq_time_step = torch.Tensor(seq_time_step).unsqueeze(2).cuda() / 180
#         else:
#             seq_time_step = torch.Tensor(seq_time_step).unsqueeze(2) / 180
#         selection_feature = 1 - self.tanh(torch.pow(self.selection_layer(seq_time_step), 2))
#         selection_feature = self.relu(self.weight_layer(selection_feature))
#         selection_feature = torch.sum(selection_feature * final_queries, 2, keepdim=True) / 8
#         selection_feature = selection_feature.masked_fill_(mask, -np.inf)
#         # time_weights = self.weight_layer(selection_feature)
#         return torch.softmax(selection_feature, 1)

In [ ]:
class Loss(nn.Module):
    def __init__(self,device):
        super(Loss, self).__init__()
        self.classify_loss = nn.BCELoss()
        self.device=device

    def forward(self, prob, labels, train=True):
        
        #prob = prob.data.cpu().numpy()
        #print(torch.sum(torch.isnan(prob)))
        pos_ind = labels >= 0.5
        neg_ind = labels < 0.5
        pos_label = labels[pos_ind]
        neg_label = labels[neg_ind]
        pos_prob = prob[pos_ind]
        neg_prob = prob[neg_ind]
        pos_loss, neg_loss = 0, 0

        
        if len(pos_prob):
            pos_prob=pos_prob.to(self.device)
            pos_label=pos_label.to(self.device)
            pos_loss = self.classify_loss(pos_prob, pos_label) 
       
        if len(neg_prob):
            neg_prob=neg_prob.to(self.device)
            neg_label=neg_label.to(self.device)
            neg_loss = self.classify_loss(neg_prob, neg_label)
        
        classify_loss = pos_loss + neg_loss
        # classify_loss = self.classify_loss(prob, labels)
        
        labels = labels.data.cpu().numpy()
        prob = prob.data.cpu().numpy()
        
        fpr, tpr, threshholds = metrics.roc_curve(labels, prob)
        auc = metrics.auc(fpr, tpr)
        
        base = ((labels==1).sum())/labels.shape[0]
        
        precision, recall, thresholds = metrics.precision_recall_curve(labels, prob)
        apr = metrics.auc(recall, precision)
        
        accur=metrics.accuracy_score(labels,prob>=0.5)
        prec=metrics.precision_score(labels,prob>=0.5)
        
        # stati number
        prob1 = prob >= 0.5
        #print(prob)
        
        pos_l = (labels==1).sum()
        neg_l = (labels==0).sum()
        pos_p = (prob1 + labels == 2).sum()#how many positives are predicted positive#####TP
        neg_p = (prob1 + labels == 0).sum()#True negatives
        prob2 = prob < 0.5
        fn    = (prob2 + labels==2).sum()
        fp    = (prob2 + labels==0).sum()
        #print(classify_loss, pos_p, pos_l, neg_p, neg_l)
        
        
        return [classify_loss, pos_p, pos_l, neg_p, neg_l, auc, apr, base, accur,prec,fn,fp,prob,labels]


In [15]:
#m = nn.Linear(3,1,True)
input = torch.randn(2, 5, 3)
input2 = torch.randn(2, 5, 1)
#output,indices = m(input)

In [16]:
input

tensor([[[-0.8795, -0.7145, -0.1966],
         [ 0.0560, -1.2541,  0.0178],
         [ 0.0141,  0.0390, -0.7342],
         [-0.0250, -1.6645,  0.2073],
         [-0.6911, -0.2338, -0.3631]],

        [[-0.9309,  0.4242, -1.0862],
         [-0.0498, -0.8568,  0.7726],
         [-1.0375, -1.3634,  0.2522],
         [-1.1481, -0.3154, -0.3021],
         [ 0.9037,  0.0808,  0.2403]]])

In [17]:
input2

tensor([[[-1.2429],
         [ 0.7973],
         [-1.1952],
         [-0.3505],
         [-0.7499]],

        [[ 1.1647],
         [ 0.4664],
         [ 0.4917],
         [-1.0107],
         [-0.0289]]])

In [18]:
torch.mul(input,input2)

tensor([[[ 1.0931,  0.8880,  0.2444],
         [ 0.0447, -0.9999,  0.0142],
         [-0.0169, -0.0466,  0.8775],
         [ 0.0088,  0.5834, -0.0726],
         [ 0.5183,  0.1753,  0.2723]],

        [[-1.0842,  0.4941, -1.2651],
         [-0.0232, -0.3996,  0.3603],
         [-0.5101, -0.6704,  0.1240],
         [ 1.1604,  0.3187,  0.3053],
         [-0.0261, -0.0023, -0.0069]]])

In [21]:
torch.sum(torch.mul(input,input2),dim=1)

tensor([[ 1.6479,  0.6003,  1.3356],
        [-0.4833, -0.2594, -0.4824]])

In [5]:
m = nn.MaxPool1d(3, stride=2)
input = torch.randn(2, 3, 5)
output = m(input)

In [6]:
input

tensor([[[-0.2419, -0.0586, -0.0050,  0.9982, -0.4716],
         [ 0.0672,  0.2972, -0.1329, -0.3279,  0.6885],
         [-0.2883,  1.4260, -0.3933,  0.2495, -1.5816]],

        [[ 0.6944,  0.5626, -0.0533,  0.4108,  1.3777],
         [ 0.4721, -0.4224,  1.0054,  0.0333, -0.0141],
         [-0.4046,  0.2370, -0.1174,  0.4251,  0.1630]]])

In [7]:
output

tensor([[[-0.0050,  0.9982],
         [ 0.2972,  0.6885],
         [ 1.4260,  0.2495]],

        [[ 0.6944,  1.3777],
         [ 1.0054,  1.0054],
         [ 0.2370,  0.4251]]])